In [1]:
from pathlib import Path
from pyraphtory.context import PyRaphtory
from pyraphtory.vertex import Vertex
from pyraphtory.spouts import FileSpout
from pyraphtory.builder import *

!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52206  100 52206    0     0   168k      0 --:--:-- --:--:-- --:--:--  172k


In [6]:
pr = PyRaphtory(logging=True).open()
rg = pr.new_graph()

13:50:16.378 [io-compute-1] INFO  com.raphtory.internals.management.Py4JServer - Starting PythonGatewayServer...
Port: 58311
Secret: cf443bf91cd3d3c4d16f78ba93ac7a8ae7941c525b13a58a29eef57ee2f1fe47
13:50:17.271 [Thread-11] INFO  com.raphtory.internals.context.LocalContext$ - Creating Service for 'simple_cornsilk_mammal'
13:50:17.290 [io-compute-6] ERROR com.raphtory.internals.management.Prometheus$ - Failed to start Prometheus on port 9999
13:50:17.303 [io-compute-6] INFO  com.raphtory.internals.management.Prometheus$ - Prometheus started on port /0:0:0:0:0:0:0:0:58313
13:50:18.327 [io-compute-6] INFO  com.raphtory.internals.components.partition.PartitionOrchestrator$ - Creating '1' Partition Managers for 'simple_cornsilk_mammal'.
13:50:19.482 [io-compute-10] INFO  com.raphtory.internals.components.partition.PartitionManager - Partition 0: Starting partition manager for 'simple_cornsilk_mammal'.


In [7]:
def parse(graph, tuple: str):
    parts = [v.strip() for v in tuple.split(",")]
    source_node = parts[0]
    src_id = graph.assign_id(source_node)
    target_node = parts[1]
    tar_id = graph.assign_id(target_node)
    time_stamp = int(parts[2])

    graph.add_vertex(time_stamp, src_id, Properties(ImmutableProperty("name", source_node)), Type("Character"))
    graph.add_vertex(time_stamp, tar_id, Properties(ImmutableProperty("name", target_node)), Type("Character"))
    graph.add_edge(time_stamp, src_id, tar_id, Type("Character_Co-occurence"))

lotr_builder = GraphB uilder(parse)
lotr_spout = FileSpout("/tmp/lotr.csv")
rg.load(Source(lotr_spout, lotr_builder))

Py4JJavaError: An error occurred while calling o3.load.
: java.lang.NullPointerException
	at com.raphtory.internals.management.QuerySender.$anonfun$submitSource$1(QuerySender.scala:96)
	at scala.collection.immutable.List.map(List.scala:246)
	at scala.collection.immutable.List.map(List.scala:79)
	at com.raphtory.internals.management.QuerySender.submitSource(QuerySender.scala:95)
	at com.raphtory.api.analysis.graphview.TemporalGraphBase.load(TemporalGraph.scala:45)
	at com.raphtory.api.analysis.graphview.TemporalGraphBase.load$(TemporalGraph.scala:44)
	at com.raphtory.api.analysis.graphview.TemporalGraph.load(TemporalGraph.scala:284)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at 

In [ ]:
cols = ["inDegree", "outDegree", "degree","triangleCount","prlabel","cclabel","twoHopPaths"]


df = rg.at(32674) \
                .past() \
                .transform(pr.algorithms.generic.centrality.PageRank())\
                .transform(pr.algorithms.generic.ConnectedComponents)\
                .transform(pr.algorithms.generic.centrality.Degree())\
                .transform(pr.algorithms.generic.TwoHopPaths())\
                .transform(pr.algorithms.generic.motif.LocalTriangleCount())\
                .execute(pr.algorithms.generic.NodeList(*cols)) \
                .write_to_dataframe(["name"] + cols)

df

13:47:04.184 [io-compute-5] INFO  com.raphtory.api.querytracker.QueryProgressTracker - Job PageRank:ConnectedComponents:Degree:TwoHopPaths:LocalTriangleCount:NodeList_4574560763518236534: Starting query progress tracker.
13:47:04.219 [spawner-akka.actor.default-dispatcher-3] INFO  com.raphtory.internals.components.querymanager.QueryManager - Query 'PageRank:ConnectedComponents:Degree:TwoHopPaths:LocalTriangleCount:NodeList_4574560763518236534' received, your job ID is 'PageRank:ConnectedComponents:Degree:TwoHopPaths:LocalTriangleCount:NodeList_4574560763518236534'.
13:47:04.230 [spawner-akka.actor.default-dispatcher-3] INFO  com.raphtory.internals.components.partition.QueryExecutor - PageRank:ConnectedComponents:Degree:TwoHopPaths:LocalTriangleCount:NodeList_4574560763518236534_0: Starting QueryExecutor.


In [5]:
pr.shutdown()

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:58255)